Detection of inconsistencies in food descriptions in online food ordering and delivery platform serves as an important ingredient for success, customer retention and satisfaction. Most companies providing online food ordering and delivery platforms are gradually utilising deep learning based solutions to detect if a food image shown on their platform conforms to the description given or category.

### Importing 

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, f1_score

In this section we will implement a food classification convolutional neural network which will be trained on food categories Benchmark dataset (Dataset description [1], [2]). The CNN architecture shown in Table 1 is a baseline model. In this section we will need to improve the model performance using minimum of three techniques presented in the course (i.e Dropout, Batch Normalization, etc.) except for transfer learning. The baseline model is to be trained on 10 epochs, using SGD optimizer, learning rate 0.001 and preprocessing of only min-max scaling. The model performance should at each training epoch should be logged to Tensorboard. The performance metrics to be monitored are training and validation loss, accuracy and F1 score. Train and test dataset can be downloaded from **torchvision.datasets.Food101.**

| Layer | Layer Type | Kernel size | Stride | Padding | Out channels |
|-------|------------|-------------|--------|---------|--------------|
| 0     | Input      | -           | -      | -       | 3            |
| 1     | Convolutional | 3 x 3     | 1      | 1 x 1   | 10           |
| 2     | Convolutional | 3 x 3     | 1      | 1 x 1   | 10           |
| 3     | Max-pooling | 2 x 2       | 2      | -       | -            |
| 5     | Convolutional | 3 x 3     | 1      | 1 x 1   | 10           |
| 6     | Convolutional | 3 x 3     | 1      | 1 x 1   | 10           |
| 7     | Max-pooling | 2 x 2       | 2      | -       | -            |
| 8     | Flatten    | -           | -      | -       | -            |
| 9     | Fully connected | -       | -      | -       | 2560 (output)|
| 10    | Fully connected (softmax) | - | - | -       | 101 (output) |


#### Step 2: Load Food101 Dataset
Load the Food101 dataset, resize images to 128x128 pixels, convert them to tensors, and normalize pixel values.

In [32]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the full train and test datasets
full_train_dataset = datasets.Food101(root='./data', split='train', download=True, transform=transform)
full_test_dataset = datasets.Food101(root='./data', split='test', download=True, transform=transform)

# Define the size of the subset (e.g., 10,000 samples)
subset_size_train = 8000
subset_size_test = 2000

# Randomly select subset indices
subset_indices_train = random.sample(range(len(full_train_dataset)), subset_size_train)
subset_indices_test = random.sample(range(len(full_test_dataset)), subset_size_test)

# Create a Subset of the train dataset using the selected indices
subset_train_dataset = Subset(full_train_dataset, subset_indices_train)

# Create a Subset of the train dataset using the selected indices
subset_test_dataset = Subset(full_test_dataset, subset_indices_test)

# Use DataLoader for the subset train dataset
subset_train_loader = DataLoader(subset_train_dataset, batch_size=32, shuffle=True)

# Use DataLoader for the full test dataset
subset_test_loader = DataLoader(subset_test_dataset, batch_size=32, shuffle=True)

#### FoodCNN Model
Define a convolutional neural network (CNN) model named `FoodCNN` for image classification.

A CNN is suitable for image classification tasks as it can automatically learn hierarchical features from images. This model consists of convolutional layers to capture spatial features, max-pooling layers to downsample the spatial dimensions, and fully connected layers for classification.

In [33]:
class FoodCNN(nn.Module):
    def __init__(self):
        super(FoodCNN, self).__init__()

        # Layer 1: Convolutional
        self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        # Layer 2: Convolutional
        self.conv2 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        # Layer 3: Max-pooling
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Layer 5: Convolutional
        self.conv3 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        # Layer 6: Convolutional
        self.conv4 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        # Layer 7: Max-pooling
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Layer 8: Flatten
        self.flatten = nn.Flatten()

        # Layer 9: Fully connected
        self.fc1 = nn.Linear(10240, 2560)
        self.relu5 = nn.ReLU()

        # Layer 10: Fully connected (softmax)
        self.fc2 = nn.Linear(2560, 101)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.max_pool1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.max_pool2(x)
        x = self.flatten(x)
        x = self.relu5(self.fc1(x))
        x = self.softmax(self.fc2(x))

        return x

# Create an instance of the model
model = FoodCNN()

#### FoodCNN2 Model
Define an improved convolutional neural network (CNN) model named `FoodCNN2` for image classification. This model includes batch normalization and dropout layers to enhance training stability and prevent overfitting.

Batch normalization normalizes the input of each layer, making the training process more stable and accelerating convergence. Dropout is used during training to randomly "drop out" units, preventing co-adaptation of hidden units and reducing overfitting.

In [34]:
import torch.nn as nn

class FoodCNN2(nn.Module):
    def __init__(self):
        super(FoodCNN2, self).__init__()

        # Layer 1: Convolutional
        self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm2d(10)
        self.dropout1 = nn.Dropout(0.2)

        # Layer 2: Convolutional
        self.conv2 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm2d(10)
        self.dropout2 = nn.Dropout(0.2)

        # Layer 3: Max-pooling
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Layer 5: Convolutional
        self.conv3 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.batch_norm3 = nn.BatchNorm2d(10)
        self.dropout3 = nn.Dropout(0.2)

        # Layer 6: Convolutional
        self.conv4 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.batch_norm4 = nn.BatchNorm2d(10)
        self.dropout4 = nn.Dropout(0.2)

        # Layer 7: Max-pooling
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Layer 8: Flatten
        self.flatten = nn.Flatten()

        # Layer 9: Fully connected
        self.fc1 = nn.Linear(10240, 2560)
        self.relu5 = nn.ReLU()
        self.batch_norm5 = nn.BatchNorm1d(2560)
        self.dropout5 = nn.Dropout(0.5)

        # Layer 10: Fully connected (softmax)
        self.fc2 = nn.Linear(2560, 101)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dropout1(self.batch_norm1(self.relu1(self.conv1(x))))
        x = self.dropout2(self.batch_norm2(self.relu2(self.conv2(x))))
        x = self.max_pool1(x)
        x = self.dropout3(self.batch_norm3(self.relu3(self.conv3(x))))
        x = self.dropout4(self.batch_norm4(self.relu4(self.conv4(x))))
        x = self.max_pool2(x)
        x = self.flatten(x)
        x = self.dropout5(self.batch_norm5(self.relu5(self.fc1(x))))
        x = self.softmax(self.fc2(x))

        return x

# Create an instance of the model
model2 = FoodCNN2()

#### FoodCNN3 Model
Define an enhanced convolutional neural network (CNN) model named `FoodCNN3` for image classification. This model incorporates deeper layers, global average pooling, and increased units in fully connected layers for improved feature extraction and discrimination.

Deeper layers allow the model to learn more complex hierarchical features. Global average pooling reduces the spatial dimensions before the fully connected layers, capturing the most important features and reducing the risk of overfitting.

In [35]:
import torch.nn as nn

class FoodCNN3(nn.Module):
    def __init__(self):
        super(FoodCNN3, self).__init__()

        # Layer 1: Convolutional
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout(0.2)

        # Layer 2: Convolutional
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout(0.2)

        # Layer 3: Max-pooling
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Layer 4: Convolutional
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout(0.2)

        # Layer 5: Convolutional
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.batch_norm4 = nn.BatchNorm2d(128)
        self.dropout4 = nn.Dropout(0.2)

        # Layer 6: Max-pooling
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Layer 7: Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Layer 8: Fully connected
        self.fc1 = nn.Linear(128, 256)
        self.relu5 = nn.ReLU()
        self.batch_norm5 = nn.BatchNorm1d(256)
        self.dropout5 = nn.Dropout(0.5)

        # Layer 9: Fully connected (softmax)
        self.fc2 = nn.Linear(256, 101)

    def forward(self, x):
        x = self.dropout1(self.batch_norm1(self.relu1(self.conv1(x))))
        x = self.dropout2(self.batch_norm2(self.relu2(self.conv2(x))))
        x = self.max_pool1(x)
        x = self.dropout3(self.batch_norm3(self.relu3(self.conv3(x))))
        x = self.dropout4(self.batch_norm4(self.relu4(self.conv4(x))))
        x = self.max_pool2(x)
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout5(self.batch_norm5(self.relu5(self.fc1(x))))
        x = self.fc2(x)

        return x

# Create an instance of the improved model
model3 = FoodCNN3()

#### Define the loss function and optimizer for training the model.
- **Loss Function (`nn.CrossEntropyLoss`):** CrossEntropyLoss is commonly used for multi-class classification problems, which is the case for food image classification. It computes the negative log likelihood of the predicted probability distribution over classes.
  
- **Optimizer (`optim.SGD`):** Stochastic Gradient Descent (SGD) is chosen as the optimizer. It's a popular optimization algorithm that updates the model parameters to minimize the loss. The learning rate (`lr`) is set to 0.001, which is a common starting value.

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

#### Define a training function to train the models.

- The `train_model` function is responsible for training the model for one epoch. It iterates over the training dataset, computes the loss, performs backpropagation, and updates the model parameters.
- The training progress is logged, including loss, accuracy, and F1 score, and the model is saved after each epoch.

In [37]:
def train_model(model, train_loader, criterion, optimizer, writer, epoch):
    model.train()
    all_labels = []
    all_predictions = []
    all_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        predicted_labels = torch.argmax(outputs, dim=1)
        all_loss += loss.item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted_labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    average_loss = all_loss / len(train_loader)
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    writer.add_scalar(f'Train {model.__class__.__name__}/Loss', average_loss, epoch)
    writer.add_scalar(f'Train {model.__class__.__name__}/Accuracy', accuracy, epoch)
    writer.add_scalar(f'Train {model.__class__.__name__}/F1_Score', f1, epoch)

    print(f'Epoch {epoch + 1}/{10}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}, Train F1 Score: {f1:.4f}')
    torch.save(model.state_dict(), f'{model.__class__.__name__}.pth')


#### Define an evaluation function to assess the performance of the models on the test set.

- The `evaluate_model` function is used to evaluate the model on the test set after training. It calculates the loss, accuracy, and F1 score.
- The evaluation results are logged, providing insights into how well the model generalizes to unseen data.

In [38]:
def evaluate_model(model, test_loader, criterion, writer, epoch):
    model.eval()
    all_predictions = []
    all_labels = []
    all_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)
            all_loss += loss.item()

    accuracy = accuracy_score(all_labels, all_predictions)
    average_loss = all_loss / len(test_loader)
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    writer.add_scalar(f'Test {model.__class__.__name__}/Loss', average_loss, epoch)
    writer.add_scalar(f'Test {model.__class__.__name__}/Accuracy', accuracy, epoch)
    writer.add_scalar(f'Test {model.__class__.__name__}/F1_Score', f1, epoch)

    print(f'Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')

#### TensorBoard Setup
Initialize TensorBoard for monitoring the model's performance during training.

In [39]:
writer = SummaryWriter(comment='task_2.1_Model_FoodCNN1')

Run the main training loop for the `FoodCNN` model on the subset of the Food101 dataset.

- The main loop iteratively trains the model on the training set (`subset_train_loader`) and evaluates its performance on the test set (`subset_test_loader`) for a specified number of epochs.
- Training and evaluation metrics are logged using Tensorboard (`writer`), providing insights into the model's performance over time.

In [40]:
# Main loop
num_epochs = 10
for epoch in range(num_epochs):
    train_model(model, subset_train_loader, criterion, optimizer, writer, epoch)
    evaluate_model(model, subset_test_loader, criterion, writer, epoch)

Epoch 1/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0003
Epoch 2/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0003
Epoch 3/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0003
Epoch 4/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0003
Epoch 5/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0003
Epoch 6/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0004
Epoch 7/10, Loss: 4.6151, Accuracy: 0.0110, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0004
Epoch 8/10, Loss: 4.6151, Accuracy: 0.0109, Train F1 Score: 0.0003
Loss: 4.6151, Accuracy: 0.0070, Test F1 Score: 0.0004
Epoch 9/10, Loss: 4.6151, Accura

#### Close Tensorboard writer

In [41]:
writer.close()

#### Load tensorboard

In [42]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir runs

#### TensorBoard Setup
Initialize TensorBoard for monitoring the model's performance during training.

In [ ]:
writer = SummaryWriter(comment='task_2.1_Model_FoodCNN2')

#### Run the main training loop for the `FoodCNN2` model on the subset of the Food101 dataset.

- The main loop iteratively trains the model on the training set (`subset_train_loader`) and evaluates its performance on the test set (`subset_test_loader`) for a specified number of epochs.
- Training and evaluation metrics are logged using Tensorboard (`writer`), providing insights into the model's performance over time.

In [43]:
# Main loop
num_epochs = 10
for epoch in range(num_epochs):
    train_model(model2, subset_train_loader, criterion, optimizer, writer, epoch)
    evaluate_model(model2, subset_test_loader, criterion, writer, epoch)

Epoch 1/10, Loss: 4.6151, Accuracy: 0.0111, Train F1 Score: 0.0112
Loss: 4.6150, Accuracy: 0.0100, Test F1 Score: 0.0080
Epoch 2/10, Loss: 4.6151, Accuracy: 0.0101, Train F1 Score: 0.0100
Loss: 4.6151, Accuracy: 0.0120, Test F1 Score: 0.0090
Epoch 3/10, Loss: 4.6151, Accuracy: 0.0103, Train F1 Score: 0.0101
Loss: 4.6151, Accuracy: 0.0115, Test F1 Score: 0.0084
Epoch 4/10, Loss: 4.6152, Accuracy: 0.0092, Train F1 Score: 0.0093
Loss: 4.6151, Accuracy: 0.0110, Test F1 Score: 0.0084
Epoch 5/10, Loss: 4.6151, Accuracy: 0.0111, Train F1 Score: 0.0110
Loss: 4.6151, Accuracy: 0.0135, Test F1 Score: 0.0100
Epoch 6/10, Loss: 4.6154, Accuracy: 0.0096, Train F1 Score: 0.0097
Loss: 4.6151, Accuracy: 0.0105, Test F1 Score: 0.0078
Epoch 7/10, Loss: 4.6153, Accuracy: 0.0095, Train F1 Score: 0.0095
Loss: 4.6151, Accuracy: 0.0110, Test F1 Score: 0.0096
Epoch 8/10, Loss: 4.6152, Accuracy: 0.0092, Train F1 Score: 0.0092
Loss: 4.6151, Accuracy: 0.0120, Test F1 Score: 0.0091
Epoch 9/10, Loss: 4.6153, Accura

#### Close Tensorboard writer

In [44]:
writer.close()

#### Load the TensorBoard notebook extension

In [45]:
%load_ext tensorboard
%tensorboard --logdir runs

#### TensorBoard Setup
Initialize TensorBoard for monitoring the model's performance during training.

In [ ]:
writer = SummaryWriter(comment='task_2.1_Model_FoodCNN3')

#### Run the main training loop for the `FoodCNN3` model on the subset of the Food101 dataset.

- The main loop iteratively trains the model on the training set (`subset_train_loader`) and evaluates its performance on the test set (`subset_test_loader`) for a specified number of epochs.
- Training and evaluation metrics are logged using Tensorboard (`writer`), providing insights into the model's performance over time.

In [46]:
# Main loop
num_epochs = 10
for epoch in range(num_epochs):
    train_model(model3, subset_train_loader, criterion, optimizer, writer, epoch)
    evaluate_model(model3, subset_test_loader, criterion, writer, epoch)

Epoch 1/10, Loss: 4.8710, Accuracy: 0.0092, Train F1 Score: 0.0072
Loss: 13.0147, Accuracy: 0.0100, Test F1 Score: 0.0060
Epoch 2/10, Loss: 4.8756, Accuracy: 0.0095, Train F1 Score: 0.0085
Loss: 13.2816, Accuracy: 0.0095, Test F1 Score: 0.0048
Epoch 3/10, Loss: 4.8786, Accuracy: 0.0100, Train F1 Score: 0.0087
Loss: 13.1440, Accuracy: 0.0095, Test F1 Score: 0.0056
Epoch 4/10, Loss: 4.8707, Accuracy: 0.0075, Train F1 Score: 0.0069
Loss: 13.4031, Accuracy: 0.0080, Test F1 Score: 0.0049
Epoch 5/10, Loss: 4.8703, Accuracy: 0.0103, Train F1 Score: 0.0094
Loss: 12.8771, Accuracy: 0.0095, Test F1 Score: 0.0056
Epoch 6/10, Loss: 4.8911, Accuracy: 0.0103, Train F1 Score: 0.0097
Loss: 13.1525, Accuracy: 0.0090, Test F1 Score: 0.0059
Epoch 7/10, Loss: 4.8713, Accuracy: 0.0115, Train F1 Score: 0.0096
Loss: 12.8194, Accuracy: 0.0080, Test F1 Score: 0.0048
Epoch 8/10, Loss: 4.8782, Accuracy: 0.0104, Train F1 Score: 0.0095
Loss: 12.6121, Accuracy: 0.0080, Test F1 Score: 0.0046
Epoch 9/10, Loss: 4.8691

#### Close Tensorboard writer

In [47]:
writer.close()

#### Load the TensorBoard notebook extension

In [48]:
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

For this section, we will need to use transfer learning. You will have to achieve better performance than the improved baseline model in before section. Suggested pretrained model is ‘DENSENET121‘ which can be loaded from torchvision.models.densenet121, however you are not only limited to ‘DENSENET121‘. The model training progress should also be monitored using using tensorboard. The performance metrics to be monitored are training and validation loss, accuracy and F1 score.

### Importing

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

#### Set device

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Define a set of data transforms using the `transforms.Compose` module from PyTorch.

- Data transforms are essential for preparing the input data for the neural network.
- In this case, the transforms include resizing the images to (224, 224) pixels, converting them to tensors, and normalizing the pixel values.

In [51]:
# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#### Load the Food101 dataset, create subsets, and use DataLoader for training and testing.

- Loading the dataset is a crucial step in preparing data for training and evaluation.
- Creating subsets allows you to work with a smaller portion of the dataset, which can be useful for testing or if the full dataset is too large.
- DataLoader is employed to efficiently load batches of data during training and testing.

In [52]:
# Load the full train and test datasets
full_train_dataset = datasets.Food101(root='./data/food-101', split='train', download=False, transform=transform)
full_test_dataset = datasets.Food101(root='./data/food-101', split='test', download=False, transform=transform)

# Define the size of the subset (e.g., 10,000 samples)
subset_size_train = 8000
subset_size_test = 2000

# Randomly select subset indices
subset_indices_train = random.sample(range(len(full_train_dataset)), subset_size_train)
subset_indices_test = random.sample(range(len(full_test_dataset)), subset_size_test)

# Create a Subset of the train dataset using the selected indices
subset_train_dataset = Subset(full_train_dataset, subset_indices_train)

# Create a Subset of the train dataset using the selected indices
subset_test_dataset = Subset(full_test_dataset, subset_indices_test)

# Use DataLoader for the subset train dataset
subset_train_loader = DataLoader(subset_train_dataset, batch_size=32, shuffle=True)

# Use DataLoader for the full test dataset
subset_test_loader = DataLoader(subset_test_dataset, batch_size=32, shuffle=True)

#### Define a transfer learning model using DENSENET121 with a custom output layer for the specified number of classes.

- Transfer learning allows leveraging pre-trained models on large datasets to boost performance on a smaller dataset.
- DENSENET121 is a deep convolutional neural network known for its effectiveness in image classification tasks.
- Modifying the fully connected layer is necessary to match the number of classes in your specific classification problem.

In [53]:
# Define DENSENET121 model with custom output layer
class TransferLearningModel(nn.Module):
    def __init__(self, num_classes):
        super(TransferLearningModel, self).__init__()
        # Load pre-trained DENSENET121 model
        self.base_model = models.densenet121(pretrained=True)
        # Freeze convolutional layers
        for param in self.base_model.parameters():
            param.requires_grad = False
        # Modify the fully connected layer for the custom number of classes
        self.base_model.classifier = nn.Sequential(
            nn.Linear(1024, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.base_model(x)

#### Create an instance of the transfer learning model and define the loss function and optimizer for training.

- Creating an instance of the model is necessary to start the training process.
- The number of classes is determined by the length of the classes in the training dataset.
- CrossEntropyLoss is commonly used for multi-class classification tasks.
- Adam optimizer is chosen for its adaptive learning rate properties.

In [54]:
# Create an instance of the model
num_classes = len(full_train_dataset.classes)
modelTransferLearningModel = TransferLearningModel(num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\progr/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [06:19<00:00, 85.3kB/s]  


#### TensorBoard Setup
Initialize TensorBoard for monitoring the model's performance during training.

In [55]:
writer = SummaryWriter(comment='task_2.2_Model_DENSENET121')

#### Train the Transfer Learning Model on the subset of the Food101 dataset and evaluate its performance on the validation set. 

- Training the model involves iterating through the training data, calculating the loss, and updating the model parameters.
- Evaluating the model on the validation set helps monitor its performance and prevent overfitting.
- Metrics such as accuracy and F1 score provide insights into the model's classification performance.

In [56]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    modelTransferLearningModel.train()
    running_loss = 0.0
    all_labels = []
    all_predictions = []

    for inputs, labels in tqdm(subset_train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = modelTransferLearningModel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    # Log metrics to Tensorboard
    writer.add_scalar('Train/Loss', running_loss / len(subset_train_loader), epoch)
    writer.add_scalar('Train/Accuracy', accuracy, epoch)
    writer.add_scalar('Train/F1_Score', f1, epoch)

    # Validation loop
    modelTransferLearningModel.eval()
    val_loss = 0.0
    all_labels_val = []
    all_predictions_val = []

    with torch.no_grad():
        for inputs_val, labels_val in tqdm(subset_test_loader, desc=f"Validation"):
            inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)

            outputs_val = modelTransferLearningModel(inputs_val)
            loss_val = criterion(outputs_val, labels_val)
            val_loss += loss_val.item()

            _, predicted_val = torch.max(outputs_val, 1)
            all_labels_val.extend(labels_val.cpu().numpy())
            all_predictions_val.extend(predicted_val.cpu().numpy())

    # Calculate metrics for validation
    accuracy_val = accuracy_score(all_labels_val, all_predictions_val)
    f1_val = f1_score(all_labels_val, all_predictions_val, average='weighted')

    # Log metrics to Tensorboard
    writer.add_scalar(f'Validation {modelTransferLearningModel.__class__.__name__}/Loss', val_loss / len(subset_test_loader), epoch)
    writer.add_scalar(f'Validation {modelTransferLearningModel.__class__.__name__}/Accuracy', accuracy_val, epoch)
    writer.add_scalar(f'Validation {modelTransferLearningModel.__class__.__name__}/F1_Score', f1_val, epoch)

Validation: 100%|██████████| 63/63 [01:45<00:00,  1.67s/it]


#### Save modelTransferLearningModel

In [59]:
torch.save(modelTransferLearningModel.state_dict(), f'{modelTransferLearningModel.__class__.__name__}.pth')

#### Close Tensorboard writer

In [57]:
writer.close()

#### Load the TensorBoard notebook extension

In [60]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...